<a href="https://colab.research.google.com/github/aeholbrook/mash_scraper/blob/main/mash_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
from bs4 import BeautifulSoup
import requests
import pandas as pd
from tqdm import tqdm 
import numpy as np
import threading

In [ ]:
def get_brew_data(href, url_base="https://www.brewersfriend.com"):


  new_resp = requests.get(url_base+href, headers ={'User-Agent': 'Mozilla/5.0'})

  soup_2 = BeautifulSoup(new_resp.content, "html.parser")

  name = soup_2.find("h3",{"itemprop":"name"})
  style = soup_2.find("span",{"class":"viewStats"})
  abv = soup_2.find("div",{"class":"abvMin"})
  ibu = soup_2.find("div",{"class":"ibuMin"})
  srm = soup_2.find("div",{"class":"srmMin"})
  score = soup_2.find("div",{"class":"reviews"})
  reviews = soup_2.find("div",{"class":"reviews"})

  if name != None: name = name.prettify().split("\n")[1][1:]
  if style != None: style = style.prettify().split("\n")[5][2:]
  if abv != None: abv = abv.prettify().split("\n")[1][1:-1]
  if ibu != None: ibu = ibu.prettify().split("\n")[1][1:]
  if srm != None: srm = srm.prettify().split("\n")[1][1:]
  if score != None: score = score.prettify().split("\n")[25][3:]
  if reviews != None: reviews = reviews.prettify().split("\n")[29][3:]
  
  df_list = pd.read_html(new_resp.text) # this parses all the tables in webpages to a list



  return { 
    "name" : name,
    "style" : style,
    "ABV" : abv,
    "IBU" : ibu,
    "SRM" : srm,
    "score" : score,
    "reviews" : reviews,
    "fermentables" : df_list[0][:-1].replace({'%':''}, regex=True),
    "hops" : df_list[1][:-1].replace({'%':''}, regex=True),
    "other" : df_list[3],
    "link" : href }

def get_hrefs(page_number, url_base="https://www.brewersfriend.com"):
  # connect to webpage
  resp = requests.get(url_base+'/homebrew-recipes/page/'+str(page_number), headers={'User-Agent': 'Mozilla/5.0'})
  # get BeautifulSoup object
  soup = BeautifulSoup(resp.content, "html.parser")

  hrefs = [item.get('href') for item in soup.find_all("a", {"class": "recipetitle"})]

  desc = "Collecting data from page " + str(page_number)

  #threads = [threading.Thread(target=get_brew_data, args)]

  data = [get_brew_data(href,url_base) for href in tqdm(hrefs, position=0, leave=False, desc = desc)]

  return data




def iterate_hrefs(start=1,stop=2):
  return np.concatenate([get_hrefs(page_number) for page_number in range(start,stop)])
  

In [ ]:
test = iterate_hrefs(1,2)

In [ ]:
test

array([{'name': 'Avg. Perfect Northeast IPA (NEIPA)', 'style': 'All Grain', 'ABV': '6.5', 'IBU': '59.26', 'SRM': '5.2', 'score': '4.79', 'reviews': '33', 'link': '/homebrew/recipe/view/363082/avg-perfect-northeast-ipa-neipa-'},
       {'name': 'Sierra Nevada Pale Ale Clone', 'style': 'All Grain', 'ABV': '5.58', 'IBU': '39.79', 'SRM': '8', 'score': '4.82', 'reviews': '28', 'link': '/homebrew/recipe/view/28546/sierra-nevada-pale-ale-clone'},
       {'name': 'Vanilla Cream Ale', 'style': 'All Grain', 'ABV': '5.48', 'IBU': '19.44', 'SRM': '4.83', 'score': '4.76', 'reviews': '17', 'link': '/homebrew/recipe/view/1633/vanilla-cream-ale'},
       {'name': 'Zombie Dust Clone - ALL GRAIN', 'style': 'All Grain', 'ABV': '5.94', 'IBU': '62.42', 'SRM': '8.5', 'score': '4.70', 'reviews': '10', 'link': '/homebrew/recipe/view/5916/zombie-dust-clone-all-grain'},
       {'name': 'Russian River Pliny the Elder (original)', 'style': 'All Grain', 'ABV': '7.09', 'IBU': '232.89', 'SRM': '6.33', 'score': '4.67